In [ ]:
import torch
from ml_eps_v_predictor import *
from pathlib import Path
import os
import numpy as np


In [ ]:
# Load the trained model
model = VE_TimeIntegrationPredictor()
model.load_state_dict(torch.load("ve_timeintegration_predictor.pth"))
model.eval()  # Set to evaluation mode
# Retrieve the normalization parameters from the dataset
mean = dataset.mean
std = dataset.std


FileNotFoundError: [Errno 2] No such file or directory: 've_timeintegration_predictor.pth'

In [ ]:

# Example new data (replace with real test values)
new_eps_t = 0.002  # Total strain
new_d_eps_t = 0.0001  # Incremental strain
new_eps_v_t = 0.0015  # Viscoelastic strain
new_d_t = 0.01  # Time step

# Convert to numpy array
new_input = np.array([new_eps_t, new_d_eps_t, new_eps_v_t, new_d_t])

# Normalize the input if the model was trained with normalized data
new_input_normalized = (new_input - mean) / (std + 1e-8)  # Avoid division by zero

# Convert to PyTorch tensor
new_input_tensor = torch.tensor(new_input_normalized, dtype=torch.float32).unsqueeze(0)  # Add batch dimension


In [ ]:
# Perform prediction
with torch.no_grad():
    predicted_eps_v_next = model(new_input_tensor).item()

print(f"Predicted next viscoelastic strain: {predicted_eps_v_next}")


In [ ]:
predicted_eps_v_next_original = (predicted_eps_v_next * std[-1]) + mean[-1]
print(f"Denormalized predicted next viscoelastic strain: {predicted_eps_v_next_original}")
